In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
import nltk
from nltk.tokenize import TreebankWordTokenizer
from random import randint
import string

In [3]:
with open("./updated_train_data.txt", 'r', encoding = 'utf8') as dataFile:
    sentences = dataFile.read().splitlines()
tokenizer = TreebankWordTokenizer()

In [4]:
#exclude words based on Wikipedia Most common words list
excluded_words = {"the", "a", "an", "be", "to", "of", "and", "in", "that", "have", "I", \
                "it", "for", "not", "on", "with", "he", "as", "you", "do", "at", "this"\
                "but", "his", "by", "from", "they", "we", "say", "her", "she", "or", "will"\
                "my", "one", "would", "there", "their", "what", "so", "up", "out",\
                "if", "about", "who", "get", "which", "go", "me", "when", "make", "can"\
                "like", "time", "no", "just", "know", "take", "into", "than", \
                 "then", "could", "how", "'", "is", '$', '£', "'s", "``", '-', '--', "'t"}
punc_set = set(string.punctuation) 
excluded_words = excluded_words | punc_set
excluded_str = ''
for item in excluded_words:
    excluded_str += item
excluded_str

'outjustofaorif.~<doanupthenhow\'sabouthavebyis*_£/foroneaswhich-=knownowhenyou,whatgothantimemewho--$&sayitwithweon!%+@#that]from)herthesotowouldcanliketake`([:>not^Iintothereandshetheir\'``};\'twillmybe|inthisbutcouldatmakeheget{they?"his\\'

In [5]:
keras_tokenizer = keras.preprocessing.text.Tokenizer(
    num_words = 50001,
    filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n``--' + excluded_str
)
keras_tokenizer.fit_on_texts(sentences)

In [6]:
#sentence-word pairs
sentence_len = 10
data = []
for sentence in sentences:
    tokenized_sent = tokenizer.tokenize(sentence.lower())
    num_groups = len(tokenized_sent)//sentence_len
    for i in range(1, num_groups):
        sent_pair = tokenized_sent[:i*sentence_len]
        word_pair = tokenized_sent[i*sentence_len]
        if word_pair not in excluded_words:
            data.append((sent_pair, word_pair))

In [7]:
#get most common words and their frequiences
unique_words = dict()
for item in data:
    if item[1] not in unique_words:
        unique_words[item[1]] = 1
    else:
        unique_words[item[1]] += 1

In [8]:
x = unique_words
dict(sorted(x.items(), key=lambda item: item[1], reverse = True))

{'was': 3916,
 'but': 3336,
 'said': 3305,
 'are': 2457,
 'has': 2434,
 'will': 2141,
 'had': 1992,
 'were': 1640,
 'been': 1597,
 'after': 1561,
 'this': 1560,
 'i': 1510,
 'its': 1504,
 'more': 1466,
 'new': 1283,
 'because': 1105,
 'can': 1070,
 'people': 1026,
 'two': 940,
 'over': 914,
 'all': 913,
 'year': 885,
 'some': 875,
 'where': 871,
 'other': 837,
 'last': 835,
 'first': 802,
 'government': 765,
 'also': 715,
 'them': 690,
 'only': 662,
 'years': 661,
 'us': 649,
 'while': 646,
 'any': 625,
 'president': 608,
 'like': 606,
 'him': 601,
 'being': 600,
 'most': 599,
 'world': 593,
 'such': 592,
 'before': 591,
 'our': 586,
 'many': 586,
 'against': 582,
 'u.s.': 580,
 'including': 564,
 'made': 541,
 'back': 534,
 'may': 533,
 'should': 532,
 'those': 524,
 'three': 523,
 'state': 521,
 'even': 519,
 'now': 513,
 'obama': 490,
 'way': 484,
 'through': 479,
 'down': 472,
 'off': 465,
 'week': 459,
 'country': 454,
 'much': 452,
 'company': 450,
 'since': 442,
 'home': 440,
 '

In [9]:
#create one-hot encoding of all the sentence-word pairs
import numpy as np
word_embedding = dict()
for i, word in enumerate(unique_words.keys()):
    embed_vector = np.zeros(shape = (1,len(unique_words)), dtype = np.int8)
    np.put(embed_vector, [i], [1])
    word_embedding[word] = embed_vector

In [10]:
print(len(unique_words))
len(word_embedding)

40000


40000

In [11]:
len(sentences)

740427

In [12]:
#create training dataset
len_training_data = int(len(data)*0.8)
len_training_data

308411

In [14]:
desired_training_data = []
for item in data:
    desired_training_data.append((item[0], word_embedding[item[1]]))


[(['the',
   'u.s.',
   'centers',
   'for',
   'disease',
   'control',
   'and',
   'prevention',
   'initially',
   'advised'],
  array([[1, 0, 0, ..., 0, 0, 0]], dtype=int8)),
 (['the',
   'u.s.',
   'centers',
   'for',
   'disease',
   'control',
   'and',
   'prevention',
   'initially',
   'advised',
   'school',
   'systems',
   'to',
   'close',
   'if',
   'outbreaks',
   'occurred',
   ',',
   'then',
   'reversed'],
  array([[0, 1, 0, ..., 0, 0, 0]], dtype=int8)),
 (['the',
   'u.s.',
   'centers',
   'for',
   'disease',
   'control',
   'and',
   'prevention',
   'initially',
   'advised',
   'school',
   'systems',
   'to',
   'close',
   'if',
   'outbreaks',
   'occurred',
   ',',
   'then',
   'reversed',
   'itself',
   ',',
   'saying',
   'the',
   'apparent',
   'mildness',
   'of',
   'the',
   'virus',
   'meant'],
  array([[0, 0, 1, ..., 0, 0, 0]], dtype=int8)),
 (['the',
   'u.s.',
   'centers',
   'for',
   'disease',
   'control',
   'and',
   'prevention',

In [15]:
np.save("./model_data", desired_training_data, allow_pickle = True)

c:\users\advai\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [17]:
loaded_data = np.load("./model_data.npy", allow_pickle = True)
loaded_data[0]

array([list(['the', 'u.s.', 'centers', 'for', 'disease', 'control', 'and', 'prevention', 'initially', 'advised']),
       array([[1, 0, 0, ..., 0, 0, 0]], dtype=int8)], dtype=object)

In [18]:
loaded_data[0][0]

['the',
 'u.s.',
 'centers',
 'for',
 'disease',
 'control',
 'and',
 'prevention',
 'initially',
 'advised']